# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы 
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным 

## 1. Загружаем данные и смотрим на них

Мы будем использовать датасет статей с Лурка, чтобы веселее было. То, что лежит на HF, кажется, не самое свежее (даже с учётом того, что Лурк де факто умер после апреля 2021 года), но материала для модели, которая будет шпарить на луркоязе, там вроде бы достаточно.

In [1]:
import pandas as pd


df = pd.read_parquet(
    "hf://datasets/averoo/lurk/data/train-00000-of-00001-75f3e3227ae9f5fc.parquet")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5671 entries, 0 to 5670
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   header             5671 non-null   object
 1   text               5671 non-null   object
 2   __index_level_0__  5671 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 133.0+ KB


In [3]:
# Пример текста
# Дисклеймер: возможен баттхёрт - не читайте
target_header = '95% населения — идиоты'
example_text = df[df['header'] == target_header]['text'].iloc[0]
print(example_text)

95% населения — идиоты

«95% населения — идиоты». Как утверждает сам Гоблин, это научный факт, по-видимому, доказанный
<del>британскими учёными</del> профессором Савельевым, из которого следует, что читающий эту статью, с вероятностью 0,95 идиот, как и её авторы.
Хотя, на самом деле, теория вероятностей не имеет дела с единичными клиническими случаями, которыми занимается психиатрия. А вот количество таких случаев — это уже, да, статистика.

 

Однако же статистикоёбы напоминают, что 95% соответствуют интервалу из двух среднеквадратичных отклонений нормально распределённой случайной величины. То есть, если взять несколько дискретных случайных величин, характеризующих идиотизм среди населения, высчитать их математическое ожидание, затем дисперсию и среднеквадратичное отклонение, потом взять интервал в два среднеквадратичных отклонения вокруг мат. ожидания, то в этот интервал попадёт 95% результатов исследования. То есть 95% населения — идиоты.
Сие предполагает двусторонний доверительный

## 2. Чистим данные

Чтобы получить более чистые данные, лучше удалить заголовок статьи в начале текста, а также возможные заголовки разделов (строки, которые не оканчиваются пунктуационным знаком)

In [4]:
import re


def remove_title(text):
    # Находим конец первой строки
    first_line_end = text.find('\n')

    # Если нет переноса строки или первая строка пустая, ничего не делаем
    if first_line_end == -1:
        return text

    first_line = text[:first_line_end].strip()

    if not first_line:
        return text

    # Проверяем, заканчивается ли строка на знак препинания
    if first_line.endswith(('.', '!', '?', ':', ';')):
        # Это не заголовок, возвращаем исходный текст
        return text
    else:
        # Это заголовок, удаляем его и перенос строки
        return text[first_line_end + 1:].lstrip()

In [5]:
df['cleaned_text'] = df['text'].apply(remove_title)

In [6]:
cleaned_example_text = df[df['header'] ==
                          target_header]['cleaned_text'].iloc[0]
print(cleaned_example_text[:200])

«95% населения — идиоты». Как утверждает сам Гоблин, это научный факт, по-видимому, доказанный
<del>британскими учёными</del> профессором Савельевым, из которого следует, что читающий эту статью, с ве


## 3. Делим на предложения и делаем сплиты: для сбора статистики и для расчёта перплексии

Тут мудрить не будем и сделаем всё с помощью razdel. Пунктуацию, увы, тоже придётся померфить.

In [7]:
from razdel import sentenize
import random
import numpy as np


all_sentences = []
for text in df['cleaned_text']:
    sentences = [s.text for s in sentenize(text) if s.text.strip()]
    all_sentences.extend(sentences)

print(f"Всего предложений: {len(all_sentences)}")

Всего предложений: 418145


In [8]:
# Перемешиваем и делим
random.seed(42)
random.shuffle(all_sentences)

# Берем 5% на тест для расчёта перплексии
split_idx = int(len(all_sentences) * 0.05)

test_sentences = all_sentences[:split_idx]
train_sentences = all_sentences[split_idx:]

print(f"Предложений в train: {len(train_sentences)}")
print(f"Предложений в test: {len(test_sentences)}")

Предложений в train: 397238
Предложений в test: 20907


## 4. Делаем триграммную модель

In [9]:
from string import punctuation
from razdel import tokenize as razdel_tokenize
from collections import Counter
from tqdm.notebook import tqdm

from scipy.sparse import lil_matrix, csc_matrix

In [10]:
# Нормализуем, как в семинаре
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word
                       in razdel_tokenize(text)]
    normalized_text = [word.lower()
                       for word in normalized_text if word and len(word) < 20]
    return normalized_text

In [11]:
# ngrammer, тоже с семинара
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0, len(tokens) - n + 1):
        ngrams.append(' '.join(tokens[i:i + n]))
    return ngrams

In [12]:
# Счётчики
unigrams = Counter()
bigrams = Counter()
trigrams = Counter()

In [13]:
# Собираем статистики по обучающей выборке
for sent in tqdm(train_sentences):
    # Добавляем два <start> токена для триграммной модели
    normalized_sent = ['<start>', '<start>'] + normalize(sent) + ['<end>']

    unigrams.update(normalized_sent)
    bigrams.update(ngrammer(normalized_sent, n=2))
    trigrams.update(ngrammer(normalized_sent, n=3))

  0%|          | 0/397238 [00:00<?, ?it/s]

In [14]:
# Создаем маппинги для слов
id2word = list(unigrams)
word2id = {word: i for i, word in enumerate(id2word)}

# Создаем маппинги для биграмм
id2bigram = list(bigrams)
bigram2id = {bigram: i for i, bigram in enumerate(id2bigram)}

In [15]:
# Матрица: строки - биграммы (контекст), столбцы - униграммы

matrix_lurk = lil_matrix((len(bigrams), len(unigrams)))

for trigram_str, trigram_count in tqdm(trigrams.items()):
    w1, w2, w3 = trigram_str.split()

    context_bigram = f"{w1} {w2}"
    target_unigram = w3

    probability = trigram_count / bigrams[context_bigram]

    matrix_lurk[bigram2id[context_bigram],
                word2id[target_unigram]] = probability

  0%|          | 0/6100940 [00:00<?, ?it/s]

In [16]:
matrix_lurk = csc_matrix(matrix_lurk)

## 5. Генерируем текст и смотрим, что получается

In [17]:
def generate_trigram(
        matrix,
        id2word,
        word2id,
        id2bigram,
        bigram2id,
        n=100,
        start_context='<start> <start>'):
    text = []
    current_context = start_context

    for i in range(n):
        # Находим ID текущей биграммы-контекста
        if current_context not in bigram2id:
            # Если мы сгенерировали биграмму, которой нет в трейне, начинаем
            # заново
            current_context = start_context

        current_context_id = bigram2id[current_context]

        # Получаем распределение вероятностей для следующего слова
        probas = matrix[current_context_id].toarray()[0]

        # Проверяем, есть ли вообще варианты
        if not np.any(probas):
            current_context = start_context
            continue

        # Нормализуем вероятности, чтобы их сумма была ровно 1.0
        probas /= probas.sum()

        # Выбираем следующее слово
        chosen_word_id = np.random.choice(matrix.shape[1], p=probas)
        chosen_word = id2word[chosen_word_id]

        text.append(chosen_word)

        if chosen_word == '<end>':
            # Если конец предложения, сбрасываем контекст
            current_context = start_context
        else:
            # Обновляем контекст
            w1, w2 = current_context.split()
            current_context = f"{w2} {chosen_word}"

    return ' '.join(text)

In [18]:
# Генерируем 5 примеров
for _ in range(5):
    generated_text = generate_trigram(
        matrix_lurk, id2word, word2id, id2bigram, bigram2id)
    print(generated_text.replace('<end>', '\n') + '\n' + '-' * 20 + '\n')

курица — « even if i know that you would face severe consequences in order to spread the truth of the night и стали выбивать эфедрин из разных ракурсов а не пошел а аргументированно отстаивал свою позицию одержимые псинобесием зоозащитники имеют о сабже 
 будучи аж пятой частью населения страны колесить по миру миллионами копий 
 в конце 80-х стали вводиться компостеры 
 в том что речи обращены в буддизм 
 суть всегда сводилась к превращению простейшего казалось бы кого ещё часто называют « смерть от естественных причин имеет склонность к киданию игроков на пк ниппонские гг-фаги написали патч позволяющий
--------------------

в час х сами они слишком заняты поглощением еды и копипасты с engadget com membrana ru а позже получили разрыв сердца 
 поэтому пластинка как оригинал а лишь одна маленькая деталь стоит на чёрном фоне 
 было открыто и смело выходят на тернистый путь сомнений и объяснений приписывает этот флаг в кустах раком » 
 при этом большинство приходящих поступать в нераспеча

Не, ну в целом-то смешно и связно, хоть и обрывается на середине и выглядит местами как просто куски из разных статей! Хотя есть несколько эпичных фразочек в стиле луркояза, пожалуй! :D

## 6. Считаем перплексию

In [19]:
# Функция перплексии из семинара
def perplexity(logp, N):
    return np.exp((-1 / N) * logp)

In [20]:
def compute_trigram_proba(text, unigrams, bigrams, trigrams):
    prob = 0
    tokens = normalize(text)
    N = len(tokens)

    if N == 0:
        return 0, 0

    # Оборачиваем в <start> и <end>
    prob_tokens = ['<start>', '<start>'] + tokens + ['<end>']

    for trigram_str in ngrammer(prob_tokens, n=3):
        w1, w2, w3 = trigram_str.split()
        context_bigram = f"{w1} {w2}"

        if context_bigram in bigrams and trigram_str in trigrams:
            # Используем логарифм частного
            prob += np.log(trigrams[trigram_str] / bigrams[context_bigram])
        else:
            # Сглаживание для неизвестных триграм
            prob += np.log(2e-5)

    return prob, N

In [21]:
# Считаем перплексию на тестовой выборке
all_ps = []
for sent in test_sentences:
    logp, N = compute_trigram_proba(sent, unigrams, bigrams, trigrams)

    if N > 0:
        all_ps.append(perplexity(logp, N))

mean_perplexity = np.mean(all_ps)
print(f"Средняя перплексия триграммной модели: {mean_perplexity}")

Средняя перплексия триграммной модели: 30427535.93038977


Жесть!

In [22]:
# А если сократить тестовую выборку до 50 предложений?

small_test_sentences = test_sentences[:50]
all_ps_small = []
for sent in small_test_sentences:
    logp, N = compute_trigram_proba(sent, unigrams, bigrams, trigrams)

    if N > 0:
        all_ps_small.append(perplexity(logp, N))
mean_perplexity_small = np.mean(all_ps_small)
print(
    f"Средняя перплексия триграммной модели на 50 предложениях: {mean_perplexity_small}")

Средняя перплексия триграммной модели на 50 предложениях: 37161.092324562225


Ну, прямо скажем — сильно меньше

In [23]:
# А на 10?

tiny_test_sentences = test_sentences[:10]
all_ps_tiny = []
for sent in tiny_test_sentences:
    logp, N = compute_trigram_proba(sent, unigrams, bigrams, trigrams)

    if N > 0:
        all_ps_tiny.append(perplexity(logp, N))
mean_perplexity_tiny = np.mean(all_ps_tiny)
print(
    f"Средняя перплексия триграммной модели на 10 предложениях: {mean_perplexity_tiny}")

Средняя перплексия триграммной модели на 10 предложениях: 19486.93981412321


Получается, что чем больше предложений в нашей выборке, на которой мы считаем перплексию, тем больше у нас получается триграмм, которые модель не может предсказать, потому что в трейне их нет, и количество шума, пропусков растёт катастрофически, из-за чего и имеем такую огромную цифру. В целом получается, что наша модель хуже предсказывает корпус, чем марковская, потому что количество ненайденных биграмм для контекста гораздо больше, чем одиночных слов.

## Задание № 2* (2 балла). 

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова. 
Сравните получаемый результат с первым заданием. 
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.

In [24]:
class Beam:
    def __init__(self, sequence: list, score: float):
        self.sequence: list = sequence
        self.score: float = score

In [25]:
def generate_trigram_beam_search(
        matrix,
        id2word,
        word2id,
        id2bigram,
        bigram2id,
        n=50,
        max_beams=5,
        prompt='<start> <start>'):

    if prompt == '<start> <start>':
        # Если промпт по умолчанию, просто начинаем с двух стартов
        normalized_prompt = ['<start>', '<start>']
    else:
        # Нормализуем промпт
        normalized_prompt = normalize(prompt)

    # Если промпт короче 2 слов, дополняем его токенами <start>
    start_tokens = (['<start>'] * (2 - len(normalized_prompt))
                    ) + normalized_prompt
    start_tokens = start_tokens[-2:]  # Оставляем ровно два токена

    initial_node = Beam(sequence=start_tokens, score=0.0)
    beams = [initial_node]

    for i in range(n):
        # Делаем n шагов генерации
        new_beams = []

        # На каждом шаге продолжаем каждый из имеющихся лучей
        for beam in beams:
            # Лучи, которые уже закончены, не продолжаем (но и не удаляем)
            if beam.sequence[-1] == '<end>':
                new_beams.append(beam)
                continue

            context_bigram = f"{beam.sequence[-2]} {beam.sequence[-1]}"

            # Если такой биграммы-контекста нет в трейне, луч отбрасывается
            if context_bigram not in bigram2id:
                continue

            context_id = bigram2id[context_bigram]

            probas = matrix[context_id].toarray()[0]

            # Если нет продолжений, луч отбрасывается
            if not np.any(probas):
                continue

            # Возьмем топ-k самых вероятных продолжений
            top_idxs = probas.argsort()[:-(max_beams + 1):-1]

            for top_id in top_idxs:
                if not probas[top_id]:
                    break

                new_word = id2word[top_id]
                new_sequence = beam.sequence + [new_word]

                new_score = beam.score + np.log(probas[top_id] + 1e-10)

                new_beam = Beam(sequence=new_sequence, score=new_score)
                new_beams.append(new_beam)

        # Сортируем все новые лучи по *нормализованному* скору
        # Делим на длину, чтобы не штрафовать длинные предложения
        beams = sorted(new_beams, key=lambda x: x.score /
                       len(x.sequence), reverse=True)[:max_beams]

    # В конце возвращаем отсортированные лучшие гипотезы
    sorted_beams = sorted(beams, key=lambda x: x.score /
                          len(x.sequence), reverse=True)

    final_sequences = []
    for beam in sorted_beams:
        final_sequence = " ".join(beam.sequence[2:])
        final_sequences.append(final_sequence)

    return final_sequences

In [26]:
# Генерируем с настройками по умолчанию, начиная с <start> <start>
default_generation = generate_trigram_beam_search(
    matrix_lurk, id2word, word2id, id2bigram, bigram2id
)
for text in default_generation:
    print(text.replace('<end>', '\n'))
    print("-" * 10)

на самом деле 

----------
на самом деле это не так уж и много чего ещё 

----------
на самом деле это не так уж и плохо 

----------
на самом деле это не так уж и много других 

----------
на самом деле это не так уж и много чего ещё не было 

----------


In [28]:
# Генерируем с промптом
prompted_gen = generate_trigram_beam_search(
    matrix_lurk, id2word, word2id, id2bigram, bigram2id,
    prompt="визуальная новелла"
)
for text in prompted_gen:
    print(f"визуальная новелла {text.replace('<end>', '\n')}")
    print("-" * 10)

визуальная новелла запиленная силами отечественных сценаристов и режиссеров 

----------
визуальная новелла запиленная силами отечественных сценаристов и продюсеров 

----------
визуальная новелла обычно требует 

----------
визуальная новелла запиленная силами отечественных сценаристов и продюсеров рубящих еврейский расовый жид 

----------
визуальная новелла запиленная силами отечественных сценаристов и продюсеров рубящих еврейский расовый жид по имени « ан стасия lifestyle d s … 

----------


Я бы сказал, что результат на самых длинных последовательностях связнее случайной генерации, но варианты по сути представляют собой просто генерации разной длины, причём не попорядку. Кроме того, похоже, они детерминированы и при перезапуске с теми же параметрами не будет ничего нового. Случайная генерация как будто выдавала более богатый текст, поэтому в смысле разговоров о конкретных улучшениях я бы остановился на ней и попытался использовать какие-то более продвинутые техники сэмплирование и игру с гиперпараметрами типа температуры.